In [ ]:
import os

import numpy as np
import pandas as pd
import PIL.Image
from IPython.display import Image
import cv2

In [ ]:
os.chdir("../")
os.getcwd()

# init stuff

In [ ]:
from keras.utils import np_utils

In [ ]:
TYPE="numbers"
# TYPE="letters"

def is_the_type(x):
#     return ord("A") <= x <= ord("Z")
    return ord("0") <= x <= ord("9")


with open("training_data/emnist-balanced-mapping.txt") as f:
    s = f.read()[:-1] # remove last char \n
a = [l.split(" ") for l in s.split("\n")]
original_mapper = {int(l[0]): chr(int(l[1])) for l in a if is_the_type(int(l[1]))}



num_classes = len(original_mapper)

to_substract = min(original_mapper.keys())

res_mapper = {k - to_substract: v for k,v in original_mapper.items()}



def filter_letters(df):
    return df[df.iloc[:,0].apply(lambda x: x in original_mapper.keys())]

print(original_mapper)



# Model training

## load dataset

In [ ]:
train_db = pd.read_csv("training_data/emnist-balanced-enriched-train.csv", header=None)
test_db  = pd.read_csv("training_data/emnist-balanced-enriched-test.csv", header = None)

In [ ]:
train_db = filter_letters(train_db)
test_db = filter_letters(test_db)

train_db.loc[:,0] = train_db[0] -  to_substract
test_db.loc[:,0] = test_db[0] -  to_substract

First, you might want to enrich the dataset - in separate notebook.

## training itself model

In [ ]:

y_train = train_db.iloc[:,0] 
y_train = np_utils.to_categorical(y_train, num_classes)


In [ ]:
from keras.utils import np_utils
from keras.layers import Input, Dense, Dropout
from keras.models import Model

y_train = train_db.iloc[:,0]
y_train = np_utils.to_categorical(y_train, num_classes)
print ("y_train:", y_train.shape)

x_train = train_db.iloc[:,1:]
x_train = x_train.astype('float32')
x_train /= 255
print ("x_train:",x_train.shape)

inp = Input(shape=(784,))
hidden_1 = Dense(1024, activation='relu')(inp)
dropout_1 = Dropout(0.2)(hidden_1)
out = Dense(num_classes, activation='softmax')(hidden_1)
model = Model(input=inp, output=out)

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(x_train, y_train, # Train the model using the training set...
          batch_size=512, nb_epoch=10,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

y_test = test_db.iloc[:,0]
y_test = np_utils.to_categorical(y_test, num_classes)
print ("y_test:", y_test.shape)

x_test = test_db.iloc[:,1:]
x_test = x_test.astype('float32')
x_test /= 255
print ("x_test:",x_train.shape)

print(model.evaluate(x_test, y_test, verbose=1)) # Evaluate the trained model on the test set!

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

## model save

In [ ]:
import json
scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
 
# serialize model to JSON
model_json = model.to_json()
with open(f"model_data/model_{TYPE}.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(f"model_data/model_{TYPE}.h5")

 
with open(f"model_data/model_{TYPE}_mapping.json", "w") as json_file:
    json.dump(res_mapper, json_file)
    

print("Saved model to disk")


# Testing the model

## model load

In [ ]:
from keras.models import model_from_json

test_db  = pd.read_csv("training_data/emnist-balanced-enriched-test.csv")
test_db = filter_letters(test_db)
y_test = test_db.iloc[:,0]
y_test = np_utils.to_categorical(y_test, num_classes)
test_db.loc[:,0] = test_db[0] -  to_substract

x_test = test_db.iloc[:,1:]
x_test = x_test.astype('float32')
x_test /= 255


# load json and create model
json_file = open('model_data/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model_data/model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy


scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

## Display data from dataset

In [ ]:
def show_letter_from_db(img):
    img = img.astype('uint8')
    img = img.reshape((28,28))
    img = img.transpose()
    img = cv2.bitwise_not(img)
    
    display(PIL.Image.fromarray(img))
    return img

In [ ]:
def revert_showing(img):
    img = cv2.bitwise_not(img)
    img = img.transpose()
    img = img.ravel()
    img = img.astype('int64') 
    return img

In [ ]:
def display_char(i):
    label = test_db.iloc[i].values[0]
    char = res_mapper.get(label)
    print("=============")
    print(f"label: {char}")
    
    img = test_db.iloc[i].values[1:]
    print("showing original")
    img_new=show_letter_from_db(img)
    
    img = revert_showing(img_new)
    
    pred = model.predict(np.array([img]))
    pred_val = res_mapper.get(pred[0].argmax())
    print(f"Prediction: {pred_val}")
    print("showing the one prediction run on")
    show_letter_from_db(img)

In [ ]:
for i in range (10):
    display_char(i)

## data from our forms

In [ ]:
from structure_parser.formstructureparser import FormStructureParser
from field_recognizer.recognize_all import recognize
from preprocessing.preprocess import preprocess

from field_recognizer.model import load_model, load_result_mapper
from main import process_document, output_data
image_path = "test/example_forms/unihack/scan1.jpg"
with open("test/example_forms/unihack/config.json", 'r') as f:
    config = json.load(f) 

fsp = FormStructureParser(config)

im = cv2.imread(image_path)

im = preprocess(im, config)
form_data = fsp.process_form(im)

In [ ]:
imgs = [im for field in form_data["fields"] for im in field["box_data"] if field["type"] ==TYPE]

In [ ]:
# preproces to training format
from field_recognizer.recognize_letters import preprocess_img, prepare_for_model_format

def preproces(img):
#     label = test_db.iloc[i].values[0]
#     char = res_mapper.get(label)ma
#     print(f"label: {char}")

    img = preprocess_img(img)
    img_m = prepare_for_model_format(img)
        
    img_m = img_m.astype('float32')
    img_m /= 255
    
    
    pred = model.predict(np.array([img_m]))
    pred_val = res_mapper.get(pred[0].argmax())
    
    conf_dict = {res_mapper.get(i):round(v,3) for i,v in enumerate(pred[0])}
    conf_dict = {k: v for k, v in sorted(conf_dict.items(), key=lambda item: item[1], reverse=True) if v>0} # sort
    print(pred_val)
    print(conf_dict)
    

    
    
    img = img.astype('uint8') 
    img = img.reshape((28,28))
    img = img.transpose()
#     display(PIL.Image.fromarray(img))

In [ ]:
for img in [img for img in imgs if len(img)>0]:
    print ("=================================")
    display(PIL.Image.fromarray(cv2.bitwise_not(img)))
    preproces(img)